<a href="https://colab.research.google.com/github/smai7/UAS-PCD/blob/main/UAS_PCD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')

Menyiapkan Direktori

In [ ]:
base_dir = '/content/drive/MyDrive/Dataset Kaki'
!ls "/content/drive/MyDrive/Dataset Kaki"

PreProcessing Data 1

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Ganti dengan path ke gambar Anda di Google Drive
image_path = '/content/drive/My Drive/Dataset Kaki/Bahan PreProcess/21-1.jpg'
# Ganti dengan path folder tujuan di Google Drive untuk menyimpan hasil
output_folder = '/content/drive/My Drive/Dataset Kaki/Hasil PreProcess/'

# Membaca gambar
image = cv2.imread(image_path)

# 1. Konversi gambar ke HSV
hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

# 2. Gaussian Blur
blurred_image = cv2.GaussianBlur(hsv_image, (5, 5), 0)

# 4. Konversi gambar ke grayscale
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# 3. Thresholding
# Konversi ke grayscale terlebih dahulu untuk thresholding
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
_, thresholded_image = cv2.threshold(gray_image, 109, 255, cv2.THRESH_BINARY)

# 5. Edge Detection menggunakan Canny
#edges = cv2.Canny(gray_image, 100, 200)

# 5. Edge Detection menggunakan Sobel
# Sobel Edge Detection
sobelx = cv2.Sobel(thresholded_image, cv2.CV_64F, 1, 0, ksize=5) # Sobel Edge Detection on the X axis
sobely = cv2.Sobel(thresholded_image, cv2.CV_64F, 0, 1, ksize=5) # Sobel Edge Detection on the Y axis
sobel_combined = cv2.sqrt(cv2.add(cv2.pow(sobelx, 2), cv2.pow(sobely, 2)))

# Konversi hasil Sobel kembali ke tipe uint8
sobel_combined = cv2.convertScaleAbs(sobel_combined)

# Menyimpan hasil preprocessing ke Google Drive
#cv2.imwrite(output_folder + '1_hsv_image.jpg', cv2.cvtColor(hsv_image, cv2.COLOR_HSV2BGR))
#cv2.imwrite(output_folder + '1_blurred_image.jpg', cv2.cvtColor(blurred_image, cv2.COLOR_HSV2BGR))
cv2.imwrite(output_folder + '21_thresholded_image.jpg', thresholded_image)
cv2.imwrite(output_folder + '21_gray_image.jpg', gray_image)
cv2.imwrite(output_folder + '21_sobel_combined.jpg', sobel_combined)

# Menampilkan hasil
plt.figure(figsize=(15, 10))

# Gambar asli
plt.subplot(2, 3, 1)
plt.title('Gambar Asli')
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.axis('off')

# Gambar HSV
plt.subplot(2, 3, 2)
plt.title('Gambar HSV')
plt.imshow(cv2.cvtColor(hsv_image, cv2.COLOR_HSV2RGB))
plt.axis('off')

# Gambar Gaussian Blur
plt.subplot(2, 3, 3)
plt.title('Gambar Gaussian Blur')
plt.imshow(cv2.cvtColor(blurred_image, cv2.COLOR_HSV2RGB))
plt.axis('off')

# Gambar Grayscale
plt.subplot(2, 3, 4)
plt.title('Gambar Grayscale')
plt.imshow(gray_image, cmap='gray')
plt.axis('off')

# Gambar Thresholding
plt.subplot(2, 3, 5)
plt.title('Gambar Thresholding')
plt.imshow(thresholded_image, cmap='gray')
plt.axis('off')

# Gambar Edge Detection (Sobel)
plt.subplot(2, 3, 6)
plt.title('Gambar Edge Detection (Sobel)')
plt.imshow(sobel_combined, cmap='gray')
plt.axis('off')

plt.show()


Machine Learning euy

In [ ]:
#Menentukan direktori ML
bahan_dir = os.path.join(base_dir,'Bahan')
train_dir = os.path.join(base_dir,'Test')
validation_dir = os.path.join(base_dir, 'Validasi')

In [ ]:
#menentukan isi direktori
arch_dir = os.path.join(bahan_dir,'Arch/')
flat_dir = os.path.join(bahan_dir,'Flat/')


print("Jumlah data tiap class")
print('Jumlah gambar telapak Arch :', len(os.listdir(arch_dir)))
print('Jumlah gambar telapak Flat :', len(os.listdir(flat_dir)))


In [ ]:
#Direktori isi test/training
train_arch = os.path.join(train_dir,'Arch/')
train_flat = os.path.join(train_dir,'Flat/')


#Direktori validasi
validation_arch = os.path.join(validation_dir,'Arch/')
validation_flat = os.path.join(validation_dir,'Flat/')


PRE PROCESSING & MENYIAPKAN DATASET

In [ ]:
import random
from shutil import copyfile

def train_val_split(source, train, val, train_ratio):
  total_size = len(os.listdir(source))
  train_size = int(train_ratio * total_size)
  val_size = total_size - train_size

  randomized = random.sample(os.listdir(source), total_size)
  train_files=randomized[0:train_size]
  val_files = randomized[train_size:total_size]

  for i in train_files:
    i_file = source + i
    destination = train + i
    copyfile(i_file, destination)

  for i in val_files:
    i_file = source + i
    destination = val + i
    copyfile(i_file, destination)

#jumlah pembagian data training dan testing
train_ratio = 0.9

#pembagian training dan validasi
#training
source_00 = arch_dir
train_00 = train_arch
val_00 = validation_arch
train_val_split(source_00, train_00, val_00, train_ratio)

#validasi
source_01 = flat_dir
train_01 = train_flat
val_01 = validation_flat
train_val_split(source_01, train_01, val_01, train_ratio)


In [ ]:
print('jumlah arch :', len(os.listdir(arch_dir)))
print('jumlah train arch :', len(os.listdir(train_arch)))
print('jumlah val arch :', len(os.listdir(validation_arch)))

In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_datagen = ImageDataGenerator(
                  rescale =1./255,
                  rotation_range = 30,
                  horizontal_flip = True,
                  shear_range = 0.3,
                  fill_mode = 'nearest',
                  width_shift_range = 0.2,
                  height_shift_range = 0.2,
                  zoom_range = 0.1
                  )

val_datagen = ImageDataGenerator(
                  rescale =1./255,
                  rotation_range = 30,
                  horizontal_flip = True,
                  shear_range = 0.3,
                  fill_mode = 'nearest',
                  width_shift_range = 0.2,
                  height_shift_range = 0.2,
                  zoom_range = 0.1
                  )

TARGET

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (666, 256),
    batch_size = 10,
    class_mode = 'categorical'
)

val_generator = val_datagen.flow_from_directory(
    validation_dir,
    target_size = (666, 256),
    batch_size = 10,
    class_mode = 'categorical'
)

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs = {}):
    if(logs.get('accuracy') > 0.99):
      print('\nAkurasi mencapai 99%')
      self.model.stop_training = True

callbacks = myCallback()

MODEL CNN

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3, 3), activation = 'relu', input_shape=(666, 256, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(200, activation = 'relu'),
    tf.keras.layers.Dropout(0.3, seed=112),
    tf.keras.layers.Dense(500, activation = 'relu'),
    tf.keras.layers.Dropout(0.5, seed=112),
    tf.keras.layers.Dense(2, activation ='sigmoid')
])


In [ ]:
model.summary()

In [ ]:
model.compile(loss = 'categorical_crossentropy',
              optimizer = 'Adam',
              metrics = ['accuracy'])

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch = 2,
    epochs = 15,
    validation_data = val_generator,
    validation_steps =1,
    verbose = 1,
    callbacks = [callbacks]
)

In [ ]:
%matplotlib inline

import matplotlib.image as mpimg
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label = 'Training Accuracy')
plt.plot(epochs, val_acc, 'b', label = 'Validation Accuracy')
plt.title('Training and Validation Accuuracy')
plt.legend(loc = 'best')
plt.show()

Mencoba Klasifikasi

In [ ]:
import numpy as np
from keras.preprocessing import image
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():

  #predicting images
  path = fn
  img = image.load_img(path, target_size = (666, 256))
  imgplot = plt.imshow(img)
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis = 0)
  x = x / 255.0

  images = np.vstack([x])
  classes = model.predict(images, batch_size = 100)

  print(fn)

  class_list = os.listdir(train_dir)

  # print(len(classes[0]))
  for j in range(len(classes[0])):
    # print(j)
    if classes[0][j] >= 0.3:
      print('Gambar ini termasuk ke', class_list[j])
    #else :
    #  print('Gambar ini termasuk ke', class_list[j])
    break
